## Testes de valência para o projeto final de IA369Y 2 Semestre 2018

Passos para tratar os dados com valência, testar e escolher um classificador para utilizar no projeto final de IA369Y.

1) Remover espaços duplos, quebras de linha, números e links do dataset e das frases a serem testadas.

2) Remover stopwords e aplicar o stemmer.

3) Treinar os classificadores.

4) Realizar as predições com os classificadores.

5) Avaliar as medidas obtidas com os classificadores.

In [1]:
import sys
sys.path.append('../..')

import csv
import codecs
import copy
import re
from random import shuffle

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk import word_tokenize
import gensim

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn.externals import joblib

from utils import tokenizer, load_six_emotions, load_3_emotions

%matplotlib inline

np.warnings.filterwarnings('ignore')
np.random.seed(12345)

def highlight_max(data, color='green'):
    attr = f'background-color: {color}; color: white; font-weight: bold;'
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

### Matriz de Resultados

In [2]:
classf = {
    'MultinomialNB': 0,
    'ComplementNB': 0,
    'LogisticRegression': 0,
    'RandomForestClassifier': 0,
    'KNeighborsClassifier': 0,
    'MLPClassifier': 0,
    'LinearSVC': 0,
    'SVC': 0
}

matriz_resultados = {
    'all': {
        'tfidf': copy.deepcopy(classf),
        'tfidf+lsa': copy.deepcopy(classf),
        'tfidf+lda': copy.deepcopy(classf),
        'count': copy.deepcopy(classf),
        'count+lsa': copy.deepcopy(classf),
        'count+lda': copy.deepcopy(classf),
        'tfidf+count+w2c': copy.deepcopy(classf),
    },
    'tweets': {
        'tfidf': copy.deepcopy(classf),
        'tfidf+lsa': copy.deepcopy(classf),
        'tfidf+lda': copy.deepcopy(classf),
        'count': copy.deepcopy(classf),
        'count+lsa': copy.deepcopy(classf),
        'count+lda': copy.deepcopy(classf),
        'tfidf+count+w2c': copy.deepcopy(classf),       
    },
    'titulo_noticias': {
        'tfidf': copy.deepcopy(classf),
        'tfidf+lsa': copy.deepcopy(classf),
        'tfidf+lda': copy.deepcopy(classf),
        'count': copy.deepcopy(classf),
        'count+lsa': copy.deepcopy(classf),
        'count+lda': copy.deepcopy(classf),
        'tfidf+count+w2c': copy.deepcopy(classf),
    }
}

## Datasets

Para validar, serão utilizados dois datasets.

Os dos datasets foram obtidos do site minerando dados.

O primeiro deles tem tweets de política de Minas Gerais com rótulos de valência: positivo, negativo e neutro. Foi feito um tratamento para eliminar tweets repetidos e dessa forma sobraram 3016 tweets.

O segundo contém 2123 títulos de notícias com rótulos de valência: positivo, negativo e neutro.

In [3]:
path = '/home/rdenadai/vagrant/python-dev/sentiment-analysis-2018-president-election/dataset/valencia/'

#Carregando os datasets
def carregar(filename):
    frases = []
    with open(filename, 'r', encoding='utf-8') as h:
        reader = csv.reader(h, delimiter='|')
        for row in reader:
            frase = tokenizer(row[0]).strip()
            valencia = row[1].upper()
            if len(frase) > 5:
                frases.append((valencia, frase))
    return frases

In [4]:
#Carrega os datasets em separado
tweets_mg = carregar(f'{path}tweets_mg_tratados.csv')
shuffle(tweets_mg)
titulo_noticias = carregar(f'{path}titulo_noticias.txt')
shuffle(titulo_noticias)

frases = tweets_mg + titulo_noticias
shuffle(frases)

print(frases[:5])
print('-' * 20)
print(tweets_mg[:5])
print('-' * 20)
print(titulo_noticias[:5])

[('POSITIVO', 'embra conclu revisã prelimin projet aér inform discussã aviaçã milit civil'), ('NEUTRO', 'adél prad venc prêmi govern min ger literatur'), ('NEGATIVO', 'esto wachins dan estar'), ('NEUTRO', 'ary antipt brasil pag cont min ger enquant govern pimentel torr dinheir bols ressac'), ('NEGATIVO', 'lut monstr dev vel faz nã transform monstr olhar durant muit temp abism abism olhar dentr')]
--------------------
[('NEUTRO', 'mais milhã pesso pun corrupçã chin estar min'), ('POSITIVO', 'pmg dupl comercializ drog centr acab delegac'), ('POSITIVO', 'detran realiz açõ educ trânsit durant mês fevereir'), ('POSITIVO', 'presídi priv min ger nunc rebeliã'), ('POSITIVO', 'soeiroanit três jov ano pres envolv roub una')]
--------------------
[('NEGATIVO', 'tod american oportun perceb potencial hom mulh esquec país nã mais esquec'), ('NEGATIVO', 'metod mau jud decid adi retorn aul motiv reduz possibil transmissã nov grip instituiçã ensin atend recomend secretari'), ('NEGATIVO', 'amér sar fer 

In [5]:
def carregar(filename):
    frases = []
    with open(filename, 'r', encoding='utf-8') as h:
        reader = csv.reader(h, delimiter='|')
        for row in reader:
            frase = ' '.join(row[0].strip().splitlines())
            valencia = row[1].upper()
            if len(frase) > 5:
                frases.append((valencia, frase))
    return frases

titulo_noticias2 = carregar(f'{path}titulo_noticias.txt')
only_neutro = []
for sent, new in titulo_noticias2:
    if sent == 'NEUTRO':
        only_neutro.append(new)
with open(f'{path}titulo_noticias_neutro.txt', 'w') as h:
    for k in only_neutro:
        h.write(f'{k}|NEUTRO\n')

In [6]:
#all_datasets
afrases = []
avalencias = []
for valencia, frase in frases:
    afrases.append(frase)
    avalencias.append(valencia)
    
print(afrases[:5])
print(avalencias[:5])
print('-' * 20)

#tweets_mg
atweets_mg = []
aval_tweets_mg = []
for valencia, frase in tweets_mg:
    atweets_mg.append(frase)
    aval_tweets_mg.append(valencia)

print(atweets_mg[:5])
print(aval_tweets_mg[:5])
print('-' * 20)

#titulo_noticias
atitulo_noticias = []
aval_titulo_noticias = []
for valencia, frase in titulo_noticias:
    atitulo_noticias.append(frase)
    aval_titulo_noticias.append(valencia)

print(atitulo_noticias[:5])
print(aval_titulo_noticias[:5])

['embra conclu revisã prelimin projet aér inform discussã aviaçã milit civil', 'adél prad venc prêmi govern min ger literatur', 'esto wachins dan estar', 'ary antipt brasil pag cont min ger enquant govern pimentel torr dinheir bols ressac', 'lut monstr dev vel faz nã transform monstr olhar durant muit temp abism abism olhar dentr']
['POSITIVO', 'NEUTRO', 'NEGATIVO', 'NEUTRO', 'NEGATIVO']
--------------------
['mais milhã pesso pun corrupçã chin estar min', 'pmg dupl comercializ drog centr acab delegac', 'detran realiz açõ educ trânsit durant mês fevereir', 'presídi priv min ger nunc rebeliã', 'soeiroanit três jov ano pres envolv roub una']
['NEUTRO', 'POSITIVO', 'POSITIVO', 'POSITIVO', 'POSITIVO']
--------------------
['tod american oportun perceb potencial hom mulh esquec país nã mais esquec', 'metod mau jud decid adi retorn aul motiv reduz possibil transmissã nov grip instituiçã ensin atend recomend secretari', 'amér sar fer divisã', 'pesso mort par armar men chanc lut nã interess tr

In [7]:
def run_ml_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    accuracy = np.round(model.score(X_test, y_test) * 100, 2)
    print(f'Modelo   : {model.__class__.__name__}')
    print(f'Acurácia : {accuracy}%')
    print('-' * 20)
    return accuracy

def split_data(X, y):
    test_size = .33
    random_state = 0
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return {
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test
    }

## Classificadores

In [8]:
classifiers = (
    MultinomialNB(),
    ComplementNB(),
    LogisticRegression(multi_class='auto', solver='lbfgs'),
    RandomForestClassifier(n_estimators=50, min_samples_split=5, random_state=0),
    KNeighborsClassifier(n_neighbors=8, algorithm='auto'),
    MLPClassifier(hidden_layer_sizes=(100, 25), max_iter=500, random_state=0),
    LinearSVC(max_iter=500),
    SVC(gamma='auto', max_iter=500),
)

## TF-IDF

In [9]:
vec_tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = vec_tfidf.fit_transform(afrases)

vec_tfidf_tmg = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf_tmg = vec_tfidf_tmg.fit_transform(atweets_mg)

vec_tfidf_tn = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf_tn = vec_tfidf_tn.fit_transform(atitulo_noticias)

In [10]:
print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf, avalencias))
        matriz_resultados['all']['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf_tmg, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf_tn, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass      


all_datasets
Modelo   : MultinomialNB
Acurácia : 58.09%
--------------------
Modelo   : ComplementNB
Acurácia : 61.6%
--------------------
Modelo   : LogisticRegression
Acurácia : 64.05%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 63.49%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 60.1%
--------------------
Modelo   : MLPClassifier
Acurácia : 62.38%
--------------------
Modelo   : LinearSVC
Acurácia : 65.81%
--------------------
Modelo   : SVC
Acurácia : 54.18%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 63.3%
--------------------
Modelo   : ComplementNB
Acurácia : 60.47%
--------------------
Modelo   : LogisticRegression
Acurácia : 65.42%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 65.22%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 61.07%
--------------------
Modelo   : MLPClassifier
Acurácia : 60.77%
--------------------
Modelo   : LinearSVC
Acurácia : 64.71%
-----------------

## LSA (usando TF-IDF)

In [11]:
#all_datasets
svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = MinMaxScaler(copy=False)
lsa = make_pipeline(svd, normalizer)
X_svd = lsa.fit_transform(X_tfidf)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, avalencias))
        matriz_resultados['all']['tfidf+lsa'][classifier.__class__.__name__] = acc
    except Exception as e:
        print(e)

#tweets_mg
X_svd = lsa.fit_transform(X_tfidf_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf+lsa'][classifier.__class__.__name__] = acc
    except:
        pass


#titulo_noticias
X_svd = lsa.fit_transform(X_tfidf_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf+lsa'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 42.9%
--------------------
Modelo   : ComplementNB
Acurácia : 55.13%
--------------------
Modelo   : LogisticRegression
Acurácia : 57.7%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 58.43%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 56.54%
--------------------
Modelo   : MLPClassifier
Acurácia : 59.59%
--------------------
Modelo   : LinearSVC
Acurácia : 58.52%
--------------------
Modelo   : SVC
Acurácia : 42.69%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 45.3%
--------------------
Modelo   : ComplementNB
Acurácia : 54.9%
--------------------
Modelo   : LogisticRegression
Acurácia : 65.82%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 63.09%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 61.07%
--------------------
Modelo   : MLPClassifier
Acurácia : 64.41%
--------------------
Modelo   : LinearSVC
Acurácia : 65.12%
------------------

## LDA (usando TF-IDF)

In [12]:
#all_datasets
lda = LatentDirichletAllocation(n_components=70, max_iter=50, random_state=0, n_jobs=5)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(lda, normalizer)
X_lda = lda.fit_transform(X_tfidf)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, avalencias))
        matriz_resultados['all']['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass


#tweets_mg
X_lda = lda.fit_transform(X_tfidf_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass


#titulo_noticias
X_lda = lda.fit_transform(X_tfidf_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 45.22%
--------------------
Modelo   : ComplementNB
Acurácia : 44.36%
--------------------
Modelo   : LogisticRegression
Acurácia : 44.96%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 46.98%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 39.9%
--------------------
Modelo   : MLPClassifier
Acurácia : 44.1%
--------------------
Modelo   : LinearSVC
Acurácia : 45.0%
--------------------
Modelo   : SVC
Acurácia : 43.24%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 51.16%
--------------------
Modelo   : ComplementNB
Acurácia : 49.34%
--------------------
Modelo   : LogisticRegression
Acurácia : 51.47%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 51.67%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 41.46%
--------------------
Modelo   : MLPClassifier
Acurácia : 49.75%
--------------------
Modelo   : LinearSVC
Acurácia : 50.76%
-----------------

## Count

In [13]:
#all_datasets
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count = vec_count.fit_transform(afrases)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count, avalencias))
        matriz_resultados['all']['count'][classifier.__class__.__name__] = acc
    except:
        pass

      
#tweets_mg
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count_tmg = vec_count.fit_transform(atweets_mg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count_tmg, aval_tweets_mg))
        matriz_resultados['tweets']['count'][classifier.__class__.__name__] = acc
    except:
        pass
      
      
#titulo_noticias
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count_tn = vec_count.fit_transform(atitulo_noticias)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count_tn, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['count'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 63.92%
--------------------
Modelo   : ComplementNB
Acurácia : 58.0%
--------------------
Modelo   : LogisticRegression
Acurácia : 65.42%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 64.35%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 31.53%
--------------------
Modelo   : MLPClassifier
Acurácia : 62.25%
--------------------
Modelo   : LinearSVC
Acurácia : 63.41%
--------------------
Modelo   : SVC
Acurácia : 45.39%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 62.39%
--------------------
Modelo   : ComplementNB
Acurácia : 60.87%
--------------------
Modelo   : LogisticRegression
Acurácia : 64.11%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 64.61%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 50.86%
--------------------
Modelo   : MLPClassifier
Acurácia : 62.69%
--------------------
Modelo   : LinearSVC
Acurácia : 63.5%
----------------

## LSA (usando Count)

In [14]:
#all_datasets
svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(svd, normalizer)
X_svd = lda.fit_transform(X_count)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, avalencias))
        matriz_resultados['all']['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass
      

#tweets_mg
X_svd = lda.fit_transform(X_count_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_tweets_mg))
        matriz_resultados['tweets']['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass
      
      
#titulos_noticias
X_svd = lda.fit_transform(X_count_tn)

print("\ntitulos_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 42.9%
--------------------
Modelo   : ComplementNB
Acurácia : 50.71%
--------------------
Modelo   : LogisticRegression
Acurácia : 57.83%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 58.9%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 55.6%
--------------------
Modelo   : MLPClassifier
Acurácia : 58.64%
--------------------
Modelo   : LinearSVC
Acurácia : 58.73%
--------------------
Modelo   : SVC
Acurácia : 45.09%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 52.68%
--------------------
Modelo   : ComplementNB
Acurácia : 59.05%
--------------------
Modelo   : LogisticRegression
Acurácia : 64.21%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 64.11%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 60.77%
--------------------
Modelo   : MLPClassifier
Acurácia : 65.93%
--------------------
Modelo   : LinearSVC
Acurácia : 65.12%
-----------------

## LDA (usando Count)

In [15]:
#all_datasets
lda = LatentDirichletAllocation(n_components=70, max_iter=50, random_state=0, n_jobs=5)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(lda, normalizer)
X_lda = lda.fit_transform(X_count)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, avalencias))
        matriz_resultados['all']['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass


#tweets_mg
X_lda = lda.fit_transform(X_count_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_tweets_mg))
        matriz_resultados['tweets']['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass
      
      
#titulo_noticias
X_lda = lda.fit_transform(X_count_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 46.46%
--------------------
Modelo   : ComplementNB
Acurácia : 45.73%
--------------------
Modelo   : LogisticRegression
Acurácia : 46.63%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 49.81%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 45.0%
--------------------
Modelo   : MLPClassifier
Acurácia : 48.26%
--------------------
Modelo   : LinearSVC
Acurácia : 46.8%
--------------------
Modelo   : SVC
Acurácia : 45.17%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 55.81%
--------------------
Modelo   : ComplementNB
Acurácia : 55.71%
--------------------
Modelo   : LogisticRegression
Acurácia : 55.81%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 57.13%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 50.15%
--------------------
Modelo   : MLPClassifier
Acurácia : 56.02%
--------------------
Modelo   : LinearSVC
Acurácia : 55.61%
----------------

## Count + TF-IDF + Word2Vec

In [16]:
#all_datasets
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(afrases)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(afrases)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in afrases:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(65078047, 70428000)

In [17]:
#all_datasets
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, avalencias))
        matriz_resultados['all']['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : LogisticRegression
Acurácia : 63.75%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 63.88%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 59.63%
--------------------
Modelo   : MLPClassifier
Acurácia : 59.16%
--------------------
Modelo   : LinearSVC
Acurácia : 64.14%
--------------------
Modelo   : SVC
Acurácia : 52.81%
--------------------


In [18]:
#tweets_mg
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(atweets_mg)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(atweets_mg)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in atweets_mg:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(21305142, 26219000)

In [19]:
#tweets_mg
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass


tweets_mg
Modelo   : LogisticRegression
Acurácia : 65.32%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 66.33%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 63.5%
--------------------
Modelo   : MLPClassifier
Acurácia : 60.87%
--------------------
Modelo   : LinearSVC
Acurácia : 62.99%
--------------------
Modelo   : SVC
Acurácia : 63.8%
--------------------


In [20]:
#titulo_noticias
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(atitulo_noticias)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(atitulo_noticias)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in atitulo_noticias:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(41546634, 44209000)

In [21]:
#titulo_noticias
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wjv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass


titulo_noticias
Modelo   : MultinomialNB
Acurácia : 66.77%
--------------------
Modelo   : ComplementNB
Acurácia : 66.62%
--------------------
Modelo   : LogisticRegression
Acurácia : 67.66%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 63.93%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 62.97%
--------------------
Modelo   : MLPClassifier
Acurácia : 65.42%
--------------------
Modelo   : LinearSVC
Acurácia : 67.88%
--------------------
Modelo   : SVC
Acurácia : 49.25%
--------------------


### Resultado dos Classificadores

#### Resultado dos Classificadores para todas as frases

In [22]:
df = pd.DataFrame.from_dict(matriz_resultados['all'])
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,61.60%,55.13%,44.36%,58.00%,50.71%,45.73%,0.00%
KNeighborsClassifier,60.10%,56.54%,39.90%,31.53%,55.60%,45.00%,59.63%
LinearSVC,65.81%,58.52%,45.00%,63.41%,58.73%,46.80%,64.14%
LogisticRegression,64.05%,57.70%,44.96%,65.42%,57.83%,46.63%,63.75%
MLPClassifier,62.38%,59.59%,44.10%,62.25%,58.64%,48.26%,59.16%
MultinomialNB,58.09%,42.90%,45.22%,63.92%,42.90%,46.46%,0.00%
RandomForestClassifier,63.49%,58.43%,46.98%,64.35%,58.90%,49.81%,63.88%
SVC,54.18%,42.69%,43.24%,45.39%,45.09%,45.17%,52.81%


#### Resultado dos Classificadores para os Tweets MG

In [23]:
df = pd.DataFrame.from_dict(matriz_resultados['tweets'])
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,60.47%,54.90%,49.34%,60.87%,59.05%,55.71%,0.00%
KNeighborsClassifier,61.07%,61.07%,41.46%,50.86%,60.77%,50.15%,63.50%
LinearSVC,64.71%,65.12%,50.76%,63.50%,65.12%,55.61%,62.99%
LogisticRegression,65.42%,65.82%,51.47%,64.11%,64.21%,55.81%,65.32%
MLPClassifier,60.77%,64.41%,49.75%,62.69%,65.93%,56.02%,60.87%
MultinomialNB,63.30%,45.30%,51.16%,62.39%,52.68%,55.81%,0.00%
RandomForestClassifier,65.22%,63.09%,51.67%,64.61%,64.11%,57.13%,66.33%
SVC,60.47%,45.30%,44.89%,44.99%,51.06%,49.04%,63.80%


#### Resultados dos Classificadores para os Títulos de Notícias

In [24]:
df = pd.DataFrame.from_dict(matriz_resultados['titulo_noticias'])
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,67.36%,57.38%,39.72%,62.30%,47.76%,41.65%,66.62%
KNeighborsClassifier,63.34%,56.86%,43.82%,49.18%,55.44%,46.57%,62.97%
LinearSVC,69.15%,63.86%,44.56%,67.59%,61.48%,46.42%,67.88%
LogisticRegression,66.54%,63.49%,44.86%,68.78%,61.25%,46.27%,67.66%
MLPClassifier,67.51%,62.00%,44.26%,66.99%,60.36%,46.13%,65.42%
MultinomialNB,65.72%,48.06%,44.78%,67.66%,47.39%,45.98%,66.77%
RandomForestClassifier,64.46%,62.74%,45.90%,65.65%,59.46%,45.16%,63.93%
SVC,51.94%,41.95%,43.29%,43.00%,43.44%,41.80%,49.25%


### Modelo escolhido e salvo

In [25]:
vec_tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = vec_tfidf.fit_transform(afrases)

svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X_svd = lsa.fit_transform(X_tfidf)

svm = LinearSVC(max_iter=1200)
model = CalibratedClassifierCV(svm) 
model.fit(X_svd, avalencias)

accuracy = np.round(model.score(X_svd, avalencias) * 100, 2)
print(f'Modelo   : {model.__class__.__name__}')
print(f'Acurácia : {accuracy}%')


filename = 'tfidf_valence.sav'
joblib.dump(vec_tfidf, filename)

filename = 'lsa_valence.sav'
joblib.dump(lsa, filename)

filename = 'model_valence.sav'
joblib.dump(model, filename)

Modelo   : CalibratedClassifierCV
Acurácia : 60.07%


['model_valence.sav']